# End Lines

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import spacy

In [3]:
from edsnlp.pipelines.core.endlines.endlinesmodel import EndLinesModel

In [4]:
import pandas as pd

In [5]:
from spacy import displacy

## Train

In [6]:
nlp = spacy.blank("fr")

In [41]:
text1 =  r"""Le patient est arrivé hier soir.
Il est accompagné par son fils

ANTECEDENTS
Il a fait une TS en 2010;
Fumeur, il est arreté il a 5 mois
Chirurgie de coeur en 2011
CONCLUSION
Il doit prendre
le medicament indiqué 3 fois par jour. Revoir médecin
dans 1 mois.
DIAGNOSTIC :

Antecedents Familiaux:
- 1. Père avec diabete

"""

In [9]:
text2 = """J'aime le \nfromage.../n"""

In [10]:
text3 = '/nIntervention(s) - acte(s) réalisé(s) :/nParathyroïdectomie élective le [DATE]'

In [43]:
texts = [text1, text2, text3,]

In [44]:
corpus =  nlp.pipe(texts)

In [45]:
endlines = EndLinesModel(nlp = nlp)

In [46]:
df = endlines.fit_and_predict(corpus)
df.head()

,original_token_index,A1,LOWER,SHAPE,IS_DIGIT,IS_SPACE,IS_UPPER,IS_PUNCT,LENGTH,DOC_ID,TEXT,SHAPE_,END_LINE,BLANK_LINE,A2,A3_,A3,A4,SPACE,SENTENCE_ID,l,mu,sigma,cv,l_norm,cv_bin,B2,l_norm_bin,B1,M1,M1_proba,M2,M2_proba,M1M2_lr,M1M2,PREDICTED_END_LINE
6,6,12646065887601541794,12646065887601541794,12646065887601541794,False,False,False,True,1,0,.,.,True,False,4524258024941853120,STRONG_PUNCT,4,5,0,0,27.0,19.846154,9.813648,0.494486,0.728969,"(0.448, 0.494]",9,"(0.423, 0.78]",4,0,0.023822,0,0.078668,0.002084,0,True
12,13,12782933457956629079,12782933457956629079,13110060611322374290,False,False,False,False,4,0,fils,xxxx,True,True,16682281906836629955,LOWER,2,6,0,1,25.0,19.846154,9.813648,0.494486,0.525171,"(0.448, 0.494]",9,"(0.423, 0.78]",4,0,0.236119,0,0.078668,0.026393,0,True
13,15,16682281906836629955,1551165722391280020,13862804199789047564,False,False,True,False,11,0,ANTECEDENTS,XXXX,True,False,4524258024941853120,UPPER,6,5,0,2,11.0,19.846154,9.813648,0.494486,-0.901413,"(0.448, 0.494]",9,"(-1.007, -0.647]",0,0,0.026399,0,0.170073,0.005556,0,True
21,24,631425121691394544,631425121691394544,631425121691394544,False,False,False,True,1,0,;,;,True,False,5632068009892776650,STRONG_PUNCT,4,5,0,3,19.0,19.846154,9.813648,0.494486,-0.086222,"(0.448, 0.494]",9,"(-0.29, 0.0666]",2,0,0.023822,0,0.102211,0.002778,0,True
30,34,17316059834606091534,17316059834606091534,13110060611322374290,False,False,False,False,4,0,mois,xxxx,True,False,7447994293974641360,LOWER,2,5,0,4,26.0,19.846154,9.813648,0.494486,0.627070,"(0.448, 0.494]",9,"(0.423, 0.78]",4,0,0.291855,0,0.078668,0.035191,0,True


In [47]:
print(df.head().to_markdown())

|    |   original_token_index |                   A1 |                LOWER |                SHAPE | IS_DIGIT   | IS_SPACE   | IS_UPPER   | IS_PUNCT   |   LENGTH |   DOC_ID | TEXT        | SHAPE_   | END_LINE   | BLANK_LINE   |                   A2 | A3_          |   A3 |   A4 |   SPACE |   SENTENCE_ID |   l |      mu |   sigma |       cv |     l_norm | cv_bin         |   B2 | l_norm_bin       |   B1 |   M1 |   M1_proba |   M2 |   M2_proba |    M1M2_lr |   M1M2 | PREDICTED_END_LINE   |
|---:|-----------------------:|---------------------:|---------------------:|---------------------:|:-----------|:-----------|:-----------|:-----------|---------:|---------:|:------------|:---------|:-----------|:-------------|---------------------:|:-------------|-----:|-----:|--------:|--------------:|----:|--------:|--------:|---------:|-----------:|:---------------|-----:|:-----------------|-----:|-----:|-----------:|-----:|-----------:|-----------:|-------:|:---------------------|
|  6 |            

In [14]:
pd.set_option("max_columns",None)

In [19]:
# Save model
PATH= "/path_to_model"
endlines.save(PATH)

## Predict

In [20]:
df2 = pd.DataFrame({"A1":[12646014,4191891561709484510 , 1668228190683662995],
                   "A2":[12646065887601541794,4191891561709484510 , 1668228190683662995],
                   "A3": ["UPPER","DIGIT","sdf"],
                   "A4": ["DIGIT","ENUMERATION","STRONG_PUNCT"],
                   "B1": [.5,.7,10.2],
                   "B2": [.0,.2,-10.2],
                  "BLANK_LINE":[False,True,False]})
df2 = endlines.predict(df2)
df2

,A1,A2,A3,A4,B1,B2,BLANK_LINE,_A3,_A4,_B1,_B2,M1,M1_proba,M2,M2_proba,M1M2_lr,M1M2,PREDICTED_END_LINE
0,12646014,12646065887601541794,UPPER,DIGIT,0.5,0.0,False,6,0,4,0,1,0.957371,0,0.135352,3.515641,1,False
1,4191891561709484510,4191891561709484510,DIGIT,ENUMERATION,0.7,0.2,True,0,1,4,3,1,0.870074,0,0.127479,0.978414,0,True
2,1668228190683662995,1668228190683662995,sdf,STRONG_PUNCT,10.2,-10.2,False,7,4,9,0,1,0.941786,1,0.556013,20.260080,1,False


## Set spans in training data (for viz)

In [21]:
set_spans = endlines.set_spans

In [22]:
set_spans(corpus, df) 

In [23]:
df.loc[df.DOC_ID==1]

,original_token_index,A1,LOWER,SHAPE,IS_DIGIT,IS_SPACE,IS_UPPER,IS_PUNCT,LENGTH,DOC_ID,TEXT,SHAPE_,END_LINE,BLANK_LINE,A2,A3_,A3,A4,SPACE,SENTENCE_ID,l,mu,sigma,cv,l_norm,cv_bin,B2,l_norm_bin,B1,M1,M1_proba,M2,M2_proba,M1M2_lr,M1M2,PREDICTED_END_LINE
67,2,13361843836202591494,13361843836202591494,4370460163704169311,False,False,False,False,2,1,le,xx,True,False,13467746054568929789,LOWER,2,2,0,0,8.0,9.0,1.414214,0.157135,-0.707107,"(0.122, 0.169]",2,"(-1.007, -0.647]",0,1,0.926865,1,0.600445,19.045213,1,False


In [24]:
doc_exemple = corpus[1]

In [25]:
doc_exemple.spans

{'new_lines': [le]}

In [26]:
doc_exemple.ents = tuple(doc_exemple.spans['new_lines'])

In [27]:
displacy.render(doc_exemple, style="ent", options={"colors":{"end_line":"green","space":"red"}})

## Pipe spacy (inference)

In [28]:
import edsnlp.components

In [29]:
nlp = spacy.blank("fr")

In [31]:
nlp.add_pipe("eds.endlines", config=dict(model_path = PATH))

In [32]:
docs2 = list(nlp.pipe([text1,text2,text3]))

In [33]:
doc_exemple = docs2[1]

In [34]:
doc_exemple

J'aime le 
fromage...

In [54]:
doc_exemple.spans["new_lines"][0].label_

'space'

In [35]:
from edsnlp.utils.filter import filter_spans
spaces = tuple(s for s in doc_exemple.spans['new_lines'] if s.label_=="space")
ents = doc_exemple.ents + spaces
ents_f = filter_spans(ents)
doc_exemple.ents = ents_f

In [36]:
displacy.render(doc_exemple, style="ent", options={"colors":{"space":"red"}})